In [2]:
## Train 
# Loading various libraries..
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split

# Reading the dataset image
img = cv2.imread('/home/aakash/opencv-4/digits.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

## Breakup the image into individual digits..
cells = [np.hsplit(row, 100) for row in np.vsplit(gray, 50)]

## Compute the HOG features for each digit and make a dataset.
dataset = []
for i in range(50):
    for j in range(100):
        feature = hog(cells[i][j], pixels_per_cell = (10, 10), cells_per_block = (1,1))
        dataset.append(feature)
dataset = np.array(dataset, 'float64')        


## Making labels for all 5k digits.
labels = []
for i in range(10):
    labels += [i]*500
    
## Using the KNN algorithm for training.
clf = KNeighborsClassifier()

x_train, x_val, y_train, y_val = train_test_split(dataset, labels, test_size = 0.3)

## Fitting the data into the classifier.
clf.fit(x_train, y_train)

## ACC Score.
print('Accuracy Score', accuracy_score(clf.predict(x_val), y_val))

## saving the classifier.
joblib.dump(clf, '/home/aakash/opencv-4/digit_classifier_knn.model')

/home/aakash/miniconda3/envs/datapysci/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


Accuracy Score 0.8873333333333333


['/home/aakash/opencv-4/digit_classifier_knn.model']

In [3]:
def load(model):
    return joblib.load(model)

def b_box(cnts):
    # get the counding rectangles.
    bounding_boxes = [cv2.boundingRect(c) for c in cnts if cv2.contourArea(c) > 300]
    # sort them left to right
    return sorted(bounding_boxes, key=lambda box: box[0])

In [4]:
def predict_digit(rect, img_thresh, clf):
    x,y,w,h = rect
    
    # expand the rectangle for better recognition.
    x = x - 10
    y -= 10
    w += 20
    h += 20
    
    roi = img_thresh[y:y+h, x:x+w]
    
    # resize the image of interest to (20,20)
    roi = cv2.resize(roi, (20,20), interpolation = cv2.INTER_AREA)
    roi = cv2.dilate(roi, (3, 3))
    
    feature = hog(roi, pixels_per_cell = (10, 10), cells_per_block = (1,1))
    digit = clf.predict([feature])
    return digit[0]

In [5]:
def perform_ocr(img, clf, write=True):
    ## preprocessing of the image.
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_gray_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)
    _, img_thresh = cv2.threshold(img_gray_blur, 90, 255, cv2.THRESH_BINARY_INV)
    
    # find the contours of the digits
    image, contours, _ = cv2.findContours(img_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
    # get the bounding rectangle for each box
    boxes = b_box(contours)
    
    reco = []
    for box in boxes:
        x,y,w,h = box
        if write:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # predict the digit inside each box
        digit = str(predict_digit(box, img_thresh, clf))
        reco.append(digit)
        
        # writing the recognized digit on image..
        if write:
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(img, digit, (x, y), font, 0.8, (255, 0, 0), 2)

    return reco

In [ ]:
## Live Testing 
img = cv2.imread('/home/aakash/opencv-4/OCR_images/2.jpg')
digits = perform_ocr(img, clf)

# making up the final number,
num = int(''.join(digits))
sqr = num**2

## Displying the information..
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(img, 'sqr({}) = {}'.format(num, sqr), (0, 40), font, 1, (255, 255, 0), 2)

cv2.imshow('Digits', img)
cv2.waitKey(0)

/home/aakash/miniconda3/envs/datapysci/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)
